In [1]:

#Basketball Dribble Counter
#Method 1 
import cv2
import numpy as np
import mediapipe as mp

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Read the video
cap = cv2.VideoCapture('C:/Users/Admin/Downloads/WHATSAAP ASSIGNMENT.mp4')

# Variables for touch count
touch_count = 0
is_touched = False
distance_sequence = []

while True:
    success, frame = cap.read()

    if not success:
        break

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the lower and upper bounds for yellow color
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Create a mask to isolate yellow regions
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Apply GaussianBlur to reduce noise and help the Hough Circle Transform
    blurred = cv2.GaussianBlur(mask, (11, 11), 0)

    # Use Hough Circle Transform to detect circles
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=40,
        param1=50,
        param2=20,
        minRadius=10,
        maxRadius=50
    )

    if circles is not None:
        # Convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")

        for (x, y, r) in circles:
            # Draw the circle in the output image
            cv2.circle(frame, (x, y), r, (0, 255, 0), 4)

            # Process the frame with Mediapipe hands
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb_frame)

            # Check if hands are detected
            if results.multi_hand_landmarks and len(results.multi_hand_landmarks) >= 2:
                # Get the middle finger landmarks for both hands
                left_hand_landmarks = results.multi_hand_landmarks[0]
                right_hand_landmarks = results.multi_hand_landmarks[1]

                # Get the middle finger landmark coordinates
                left_middle_finger = left_hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                right_middle_finger = right_hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

                # Convert normalized coordinates to pixel coordinates
                height, width, _ = frame.shape
                left_middle_finger_x, left_middle_finger_y = int(left_middle_finger.x * width), int(left_middle_finger.y * height)
                right_middle_finger_x, right_middle_finger_y = int(right_middle_finger.x * width), int(right_middle_finger.y * height)

                # Calculate distance between the hands and the circle center
                distance_left = np.sqrt((left_middle_finger_x - x)**2 + (left_middle_finger_y - y)**2)
                distance_right = np.sqrt((right_middle_finger_x - x)**2 + (right_middle_finger_y - y)**2)

                # Display distance on the frame with resized font
                font_size = 0.8
                cv2.putText(frame, f'Distance to Left Hand: {distance_left:.2f}', (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.putText(frame, f'Distance to Right Hand: {distance_right:.2f}', (50, 80),
                            cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 255, 255), 2, cv2.LINE_AA)

                # Find the minimum distance
                min_distance = min(distance_left, distance_right)

                # Update the distance sequence
                distance_sequence.append(min_distance)

                # Display the minimum distance on the frame
                cv2.putText(frame, f'Minimum Distance: {min_distance:.2f}', (50, 110),
                            cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 255, 255), 2, cv2.LINE_AA)

                # Check if the hand and circle touch
                if min_distance < r:
                    if not is_touched:
                        is_touched = True
                else:
                    is_touched = False

                # Check if the sequence of minimum distances is less than 100
                if len(distance_sequence) >= 3 and all(d < 100 for d in distance_sequence[-3:]):
                    # Increase the touch count
                    touch_count += 1
                    # Reset the distance sequence
                    distance_sequence = []

        # Display touch count on the frame with resized font
        cv2.putText(frame, f'Touch Count: {touch_count}', (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Object Detection', frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(85) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
#work_ball count

In [2]:
#Basketball Dribble Counter
#Method 2
import cv2
from ultralytics import YOLO
import numpy as np

class DribbleCounter:
    def __init__(self, video_path):
        # Load the YOLO model for ball detection
        self.model = YOLO("C:/Users/Admin/Downloads/basketballModel.pt")

        # Open the video file
        self.cap = cv2.VideoCapture(video_path)

        # Initialize variables to store the previous position of the basketball
        self.prev_x_center = None
        self.prev_y_center = None
        self.prev_delta_y = None

        # Initialize the dribble counter
        self.dribble_count = 0

        # Threshold for the y-coordinate change to be considered as a dribble
        self.dribble_threshold = 10

    def run(self):
        # Process frames from the video until the end
        while self.cap.isOpened():
            success, frame = self.cap.read()
            if success:
                self.process_frame(frame)

                if cv2.waitKey(1) & 0xFF == ord("q"):
                    break
            else:
                break

        # Release the video capture and destroy the window
        self.cap.release()
        cv2.destroyAllWindows()

    def process_frame(self, frame):
        results_list = self.model(frame, verbose=False, conf=0.50)

        for results in results_list:
            for bbox in results.boxes.xyxy:
                x1, y1, x2, y2 = bbox[:4]

                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2

                print(f"Ball coordinates: (x={x_center:.2f}, y={y_center:.2f})")

                self.update_dribble_count(x_center, y_center)

                self.prev_x_center = x_center
                self.prev_y_center = y_center

            annotated_frame = results.plot()

            # Draw the dribble count on the frame
            cv2.putText(annotated_frame, f'Dribble Count: {self.dribble_count}', (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.imshow("YOLOv8 Inference", annotated_frame)

    def update_dribble_count(self, x_center, y_center):
        if self.prev_y_center is not None:
            delta_y = y_center - self.prev_y_center

            if (
                self.prev_delta_y is not None
                and self.prev_delta_y > self.dribble_threshold
                and delta_y < -self.dribble_threshold
            ):
                self.dribble_count += 1

            self.prev_delta_y = delta_y

if __name__ == "__main__":
    video_path = "C:/Users/Admin/Downloads/WHATSAAP ASSIGNMENT.mp4"
    dribble_counter = DribbleCounter(video_path)
    dribble_counter.run()
#10-97

Ball coordinates: (x=212.49, y=245.45)
Ball coordinates: (x=211.68, y=263.43)
Ball coordinates: (x=207.34, y=320.41)
Ball coordinates: (x=184.71, y=74.64)
Ball coordinates: (x=184.07, y=74.81)
Ball coordinates: (x=183.42, y=74.94)
Ball coordinates: (x=191.43, y=559.12)
Ball coordinates: (x=211.80, y=319.88)
Ball coordinates: (x=214.81, y=302.55)
Ball coordinates: (x=217.17, y=300.61)
Ball coordinates: (x=217.36, y=309.83)
Ball coordinates: (x=217.29, y=329.39)


In [3]:
#Trajectory of the Ball 
import cv2
import numpy as np

# Replace 'your_video_path.mp4' with the actual path to your video file
video_path = 'C:/Users/Admin/Downloads/WHATSAAP ASSIGNMENT.mp4'

# If a video path was not supplied, use the webcam
if not video_path:
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture(video_path)

# Set the desired frame size for visibility on the laptop screen
visible_width = 640
visible_height = 480

# Set the frame width and height
cap.set(cv2.CAP_PROP_FRAME_WIDTH, visible_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, visible_height)

# Initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_path = 'output_trajectory.avi'
out = cv2.VideoWriter(output_path, fourcc, 20.0, (visible_width, visible_height))

# Define color range for the basketball
colourLow = (0, 100, 50)
colourHigh = (20, 255, 255)

# Initialize variables for trajectory detection
trajectory_points = []

while True:
    ret, frame = cap.read()

    if frame is None:
        break

    # Resize the frame to the desired visible size
    frame = cv2.resize(frame, (visible_width, visible_height))

    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, colourLow, colourHigh)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None

    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        if radius > 15 and cv2.contourArea(c) > 150:
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)

            # Add the current center to the trajectory points
            trajectory_points.append(center)

            # Draw the trajectory on the frame
            for i in range(1, len(trajectory_points)):
                cv2.line(frame, trajectory_points[i - 1], trajectory_points[i], (0, 255, 0), 2)

    out.write(frame)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
#work_1_trajectory 

In [5]:
#Optical Flow of Objects 
import cv2
import numpy as np

# Replace 'your_video_path.mp4' with the actual path to your video file
video_path = 'C:/Users/Admin/Downloads/WHATSAAP ASSIGNMENT.mp4'

# If a video path was not supplied, use the webcam
if not video_path:
    cap = cv2.VideoCapture(0)
else:
    cap = cv2.VideoCapture(video_path)

# Take the first frame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# Define a random set of points in the first frame
p0 = cv2.goodFeaturesToTrack(old_gray, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

while True:
    ret, frame = cap.read()
    if frame is None:
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(
        old_gray, frame_gray, p0, None, winSize=(15, 15), maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
    )

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)
        c, d = old.ravel().astype(int)
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
        frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)

    cv2.imshow('Optical Flow', img)

    k = cv2.waitKey(30) & 0xff
    if k == 27:  # Press 'Esc' to exit
        break

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.destroyAllWindows()
#worker_opticalflow

In [1]:
#Automatically Detecting Outlines And Borders of Objects
import os
import cv2
import tkinter as tk
from tkinter import messagebox
import numpy as np
import random

class VideoProcessor:
    def __init__(self, root):
        self.root = root
        self.root.title("Video Processor")

        self.instructions = [
            "1. After clicking 'Run Video Processing,' press 's' to pause the video.",
            "2. After pressing 's,' use the mouse to click and crop the image, then press 'Enter.'",
            "3. After pressing 'Enter,' you can view the cropped image.",
            "4. After viewing the cropped image, press 'c' to clear and repeat the process of clicking and cropping the image to view the cropped image again.",
            "5. Only when you are viewing the cropped image can you quit the repeating process by pressing 'q.'",
            "6. After pressing 'q,' you will return to resume the video that was paused, and you can repeat the process."
        ]

        self.instructions_label = tk.Label(root, text="\n".join(self.instructions), justify='left')
        self.instructions_label.pack()

        self.run_button = tk.Button(root, text="Run Video Processing", command=self.run_video_processing)
        self.run_button.pack()

        self.exit_button = tk.Button(root, text="Exit", command=self.root.destroy)
        self.exit_button.pack()

    def run_video_processing(self):
        messagebox.showinfo("Video Processor", "Video processing started. Press 's' to pause the video.")
        self.root.update_idletasks()  # Update the Tkinter window

        input_video_path = "C:/Users/Admin/Downloads/WHATSAAP ASSIGNMENT.mp4"
        cap = cv2.VideoCapture(input_video_path)

        if not cap.isOpened():
            messagebox.showerror("Video Processor", "Error opening video")
            return

        while True:
            ret, frame = cap.read()
            if not ret:
                messagebox.showerror("Video Processor", "Video frame completed ")
                break

            cv2.imshow('Video Viewer', frame)
            self.root.update_idletasks()  # Update the Tkinter window

            key = cv2.waitKey(1) & 0xFF

            if key == ord('s'):
                self.user_interactions(frame, cap)
            elif key == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    def user_interactions(self, img, cap):
        original_img = img.copy()

        while True:
            roi = cv2.selectROI(img)
            cv2.destroyAllWindows()

            x, y, w, h = roi

            if x < 0 or y < 0 or x + w > img.shape[1] or y + h > img.shape[0]:
                print("Invalid ROI. Please try again.")
                img = original_img.copy()
                continue

            cropped_img = img[y:y+h, x:x+w]

            # Perform image processing techniques (resize, brighten, etc.)
            processed_img = self.process_cropped_image(cropped_img)

            # Display the processed image
            cv2.imshow('Processed Cropped Image', processed_img)

            key = cv2.waitKey(0) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('c'):
                img = original_img.copy()

            # Save the processed image with a random number in the name
            save_dir = 'E:/computer_vision_intern/images/'
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)

            try:
                img_name = f'processed_cropped_image_{random.randint(1, 100000)}.jpg'
                full_path = os.path.join(save_dir, img_name)
                cv2.imwrite(full_path, processed_img)

                print(f"Saved image: {full_path}")

            except Exception as e:
                print(f"Error saving image: {str(e)}")

        cv2.destroyAllWindows()

    def process_cropped_image(self, img):
        # Resize the image to (216, 216)
        resized_img = cv2.resize(img, (416, 416))

        # Brighten the image
        brightness_factor = 1.5
        brightened_img = cv2.convertScaleAbs(resized_img, alpha=brightness_factor, beta=0)

        return brightened_img

if __name__ == "__main__":
    root = tk.Tk()
    video_processor = VideoProcessor(root)
    root.mainloop()
